In [1]:
from generator import Croupier

Импортируем класс Croupier, который отвечает за генерацию данных и создадим экземпляр класса с заданными значениями вероятности для нечестного кубика, а так же вероятности перехода с честного кубика на нечестный и на себя самого, и вероятности перехода с нечестного кубика на себя и на честный. Проверим что получаем выпавшие знаечение кубика, а так же метку кубика

In [24]:
casino = Croupier([0.1,0.1,0.1,0.1,0.1,1/2],[0.05,0.95],[0.9,0.1])
for i in range(10):
    print(casino())

(1, 'F')
(6, 'F')
(2, 'U')
(3, 'U')
(1, 'U')
(2, 'U')
(4, 'U')
(2, 'F')
(3, 'F')
(1, 'F')


Сгенерируем датасеты размером 1 000, 10 000 и 100 000

In [29]:
dataset1000 = []
dataset1000Labels = []

for i in range(1000):
    data, label = casino()
    dataset1000.append(data)
    dataset1000Labels.append(label)

In [30]:
dataset10000 = []
dataset10000Labels = []

for i in range(10000):
    data, label = casino()
    dataset10000.append(data)
    dataset10000Labels.append(label)

In [31]:
dataset100000 = []
dataset100000Labels = []

for i in range(100000):
    data, label = casino()
    dataset100000.append(data)
    dataset100000Labels.append(label)

Определим базовый класс алгоритма

In [ ]:
class baseAlgo():
    def __init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None):
        self.data = data
        self.L = len(data)
        
        if fairDice == None: self.fairDice = [1/6 for i in range(6)]
        else: self.fairDice = fairDice
        self.unfairDice = unfairDice
        self.fairDiceStateProbability = fairDiceStateProbability
        self.unfairDiceStateProbability = unfairDiceStateProbability
        
        self.path = []

In [25]:
class algorithmViterbi(baseAlgo):
    def __init__(self, data):
        baseAlgo.__init__(self)
        self.v = [0 for i in range(self.L)]
        self.v[0] = 1
        
    def step(self, index):
        pass
    
    def __call__(self):
        # Рекурсия
        for i in range(1,self.L):
            self.step(i)
        # Завершение
        pass
        # Обратный ход
        pass

In [26]:
class forwardAlgorithm(baseAlgo):
    def __init__(self, data):
        baseAlgo.__init__(self,data)
        pass
    
    def step(self):
        pass
    
    def __call__(self):
        pass

In [27]:
class backwardAlgorithm(baseAlgo):
    def __init__(self, data):
        baseAlgo.__init__(self,data)
        pass
    
    def step(self):
        pass
    
    def __call__(self):
        pass 

In [28]:
class BaumWelchTraining():
    pass